In [1]:
def quantify_results(Method, direc, SaveName, frameresult, OrigFrameFolder, OrigAnnotFolder):
        from tqdm.notebook import tqdm, trange
        from matplotlib.pyplot import imshow, subplots
        import cv2
        import numpy as np
        import glob
        import os
        import pandas as pd
        from skimage.measure import compare_ssim as ssim
        from skimage.measure import compare_mse as mse
        from skimage.measure import compare_psnr as psnr
        from PIL import Image

        #for cross change all ellipse to cross
        columnNames=["videoName", "Method", "PSNRavg","PSNRstd", "SSIMavg","SSIMstd", "MSEavg","MSEstd","PSNRCropavg","PSNRCropstd", "SSIMCropavg","SSIMCropstd", "SSIMCropFullavg","SSIMCropFullstd", "MSECropavg","MSECropstd"]
        df = pd.DataFrame(columns=columnNames)            

        videoNames=[f'hyperK_{i}' for i in range(343, 373)]



        for videoName in tqdm(videoNames):
                MaskFile=OrigAnnotFolder+videoName+"_TransMask/"
                TransFrameFiles=sorted(glob.glob(direc+videoName+frameresult+"/*"))
                OrigFrameFile=OrigFrameFolder+videoName+"/"
                PSNR=[]
                MSE=[]
                SSIM=[]
                PSNRCrop=[]
                MSECrop=[]
                SSIMCrop=[]
                SSIMCropFull=[]
                cnt=0
                adder=0
                for TransFrameFile in TransFrameFiles:
                        cnt=cnt+1
                        numStr=int(''.join(filter(str.isdigit, os.path.basename(TransFrameFile))))
                        if cnt==1 and numStr==1: adder=1
                        else: adder=0
                        TransName=(str(numStr-adder)).zfill(5)+".jpg"
                        TransImage= np.array(Image.open(TransFrameFile))
                        MaskImage=Image.open(MaskFile+TransName)
                        MaskImage = np.array(MaskImage.convert('L'))
                        MaskImage = np.array(MaskImage > 199).astype(np.uint8) #Rema:from 0 to 199 changes to binary better
                        MaskImage = cv2.dilate(MaskImage, cv2.getStructuringElement(
                                cv2.MORPH_ELLIPSE, (8, 8)), iterations=1) #Rema:Dilate only 1 iteration
                        if np.size(np.where(MaskImage==1))>=30:
                                OrigImage=np.array(Image.open(OrigFrameFile+TransName))

                                wm, hm = MaskImage.shape
                                MaskImageRGB = np.empty((wm, hm, 3), dtype=np.uint8)
                                MaskImageRGB[:, :, 2] =  MaskImageRGB[:, :, 1] =  MaskImageRGB[:, :, 0] =  MaskImage

                                TransImageFull=TransImage*MaskImageRGB
                                OrigImageFull=OrigImage*MaskImageRGB

                                PSNR.append(psnr(OrigImage,TransImageFull))
                                MSE.append(mse(OrigImage,TransImageFull))
                                SSIM.append(ssim(OrigImage,TransImageFull,multichannel=3))

                                TransImageCrop=TransImage[MaskImage!=0]
                                OrigImageCrop=OrigImage[MaskImage!=0]

                                PSNRCrop.append(psnr(OrigImageCrop,TransImageCrop))
                                MSECrop.append(mse(OrigImageCrop,TransImageCrop))

                                #ssim with full gives mmsim, S
                                S=ssim(OrigImage,TransImage,multichannel=3,full=True)
                                S=S[1]
                                S=np.mean(S,axis=2)
                                Scrop=S[MaskImage!=0]
                                Scrop_mean=np.mean(Scrop,axis=0)
                                SSIMCropFull.append(Scrop_mean)

                                SSIMCrop.append(ssim(OrigImageCrop,TransImageCrop,multichannel=3))


                PSNRavg=np.mean(PSNR)
                PSNRstd=np.std(PSNR)

                MSEavg=np.mean(MSE)
                MSEstd=np.std(MSE)

                SSIMavg=np.mean(SSIM)
                SSIMstd=np.std(SSIM)

                PSNRCropavg=np.mean(PSNRCrop)
                PSNRCropstd=np.std(PSNRCrop)

                MSECropavg=np.mean(MSECrop)
                MSECropstd=np.std(MSECrop)

                SSIMCropavg=np.mean(SSIMCrop)
                SSIMCropstd=np.std(SSIMCrop)

                SSIMCropFullavg=np.mean(SSIMCropFull)
                SSIMCropFullstd=np.std(SSIMCropFull)

                df_temp=pd.DataFrame([[videoName, Method,
                                PSNRavg, PSNRstd, SSIMavg,SSIMstd, MSEavg,MSEstd,
                                PSNRCropavg, PSNRCropstd, SSIMCropavg, SSIMCropstd, 
                                SSIMCropFullavg, SSIMCropFullstd, MSECropavg, MSECropstd]],columns=columnNames)
                df=df.append(df_temp)

        df.to_csv(SaveName)  

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
frameresult="/frameresult/"
OrigFrameFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"
OrigAnnotFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/Annotations_TransMask/"
for Method in  ["TransMask_8Dil_Bin_pretrained_ellipse",
    "TransMask_transfer2_trainTransMask_gen_00005ellipse" ,
    "TransMask_transfer2_trainTransMask_gen_00008ellipse" ,
    "TransMask_transfer2_trainTransMask_gen_00006ellipse" ,
    "TransMask_transfer2_trainTransMask_gen_00009ellipse" ,
    "TransMask_trainTransMask_gen_00002ellipse" ,
    "TransMask_trainTransMask_gen_00011ellipse" , 
    "TransMask_transfer2_trainTransMask_gen_00009ellipse_NoTemporal"]:
    direc="examples/hyper-kvasir/"+Method+"/"
    SaveName=r"Results19-9-2922/Results"+Method+".csv"
    
    quantify_results(Method, direc, SaveName, frameresult, OrigFrameFolder)
    

# Examples

In [ ]:
# make sure in test code :
# #added for memory issue
# if len(frames)>927:
#     masks=masks[:927]
#     frames=frames[:927]

import glob
import os
for videoDir in glob.glob("/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/testFrames/*"):
    videoName = os.path.basename(videoDir)
    cmd = ["python test-cycle.py --gpu 7 " +
        "--NoMask --output /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/ " +
        "--frame /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"+videoName+"/ " +
        "--mask /home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testAnnot/"+videoName+"/ " +
        "--ckptpath /home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/sttn_SanityCheck16/ " +
        "--ckptnumber 20"][0]
    print(cmd)
    os.system(cmd)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
frameresult="/nomask/A/frameresult/"
OrigFrameFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/testFrames/"
OrigAnnotFolder = "/home/datasets/Hyper_Kvasir/Resized_Renamed_Images_withMasks/Data_NotCropped_usedForTrainingEndoSTTNoriginally/Annotations_TransMask/"
Method = "sttn_SanityCheck16"
iter="gen_00020"
direc="/home/datasets/Trainedmodelsandresults/Endo-STTN/SanityChecks/"+Method+"/"+iter+"/"
SaveName=direc+"ResultsInpainting"+Method+".csv"

quantify_results(Method, direc, SaveName, frameresult, OrigFrameFolder)